In [1]:
!nvidia-smi

Sun Oct  9 17:25:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls -alh /mydrive/yolov3 

Mounted at /content/gdrive
total 419M
drwx------ 2 root root 4.0K Oct  9 13:28 colab_0
drwx------ 2 root root 4.0K Oct  9 17:13 colab_0_restart
-rw------- 1 root root  54M Oct  9 01:56 images0.zip
-rw------- 1 root root  66M Oct  9 17:12 images1.zip
-rw------- 1 root root  66M Oct  9 01:55 images.zip
-rw------- 1 root root 8.2K Oct  9 16:26 yolov3_training.cfg
-rw------- 1 root root 235M Oct  9 17:14 yolov3_training_final_0.weights


In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15494, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 15494 (delta 0), reused 2 (delta 0), pack-reused 15490
Receiving objects: 100% (15494/15494), 14.11 MiB | 11.98 MiB/s, done.
Resolving deltas: 100% (10408/10408), done.


In [4]:
%cd darknet

/content/darknet


In [5]:
!ls -alh .

total 272K
drwxr-xr-x 14 root root 4.0K Oct  9 17:26 .
drwxr-xr-x  1 root root 4.0K Oct  9 17:26 ..
drwxr-xr-x  4 root root 4.0K Oct  9 17:26 3rdparty
drwxr-xr-x  3 root root 4.0K Oct  9 17:26 build
-rwxr-xr-x  1 root root  35K Oct  9 17:26 build.ps1
drwxr-xr-x  3 root root 4.0K Oct  9 17:26 cfg
drwxr-xr-x  2 root root 4.0K Oct  9 17:26 .circleci
drwxr-xr-x  3 root root 4.0K Oct  9 17:26 cmake
-rw-r--r--  1 root root  25K Oct  9 17:26 CMakeLists.txt
-rw-r--r--  1 root root 1.4K Oct  9 17:26 DarknetConfig.cmake.in
-rw-r--r--  1 root root 9.4K Oct  9 17:26 darknet_images.py
-rw-r--r--  1 root root  11K Oct  9 17:26 darknet.py
-rw-r--r--  1 root root 6.6K Oct  9 17:26 darknet_video.py
drwxr-xr-x  3 root root 4.0K Oct  9 17:26 data
drwxr-xr-x  8 root root 4.0K Oct  9 17:26 .git
drwxr-xr-x  4 root root 4.0K Oct  9 17:26 .github
-rw-r--r--  1 root root  686 Oct  9 17:26 .gitignore
-rwxr-xr-x  1 root root  110 Oct  9 17:26 image_yolov3.sh
-rwxr-xr-x  1 root root  110 Oct  9 17:26 image_yolov4

In [6]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [7]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!cat data/obj.data

classes= 2
train  = data/train.txt
valid  = data/test.txt
names = data/obj.names
backup = /mydrive/yolov3


In [8]:
# 2 classes, filters = (num_classes+5)*3 = 21
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

In [9]:
!echo "Gumby" > data/obj.names
!echo "Pokey" >> data/obj.names
!cat data/obj.names

Gumby
Pokey


In [10]:
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj
!cat data/obj.data

classes= 2
train  = data/train.txt
valid  = data/test.txt
names = data/obj.names
backup = /mydrive/yolov3


In [11]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/15641_1.jpg    
  inflating: data/obj/15641_1.txt    
  inflating: data/obj/clay-gumby-twisted.jpg  
  inflating: data/obj/clay-gumby-twisted.txt  
 extracting: data/obj/g0_Color.jpg   
  inflating: data/obj/g0_Color.txt   
 extracting: data/obj/g10_Color.jpg  
  inflating: data/obj/g10_Color.txt  
 extracting: data/obj/g11_Color.jpg  
  inflating: data/obj/g11_Color.txt  
 extracting: data/obj/g12_Color.jpg  
 extracting: data/obj/g12_Color.txt  
 extracting: data/obj/g13_Color.jpg  
  inflating: data/obj/g13_Color.txt  
 extracting: data/obj/g14_Color.jpg  
  inflating: data/obj/g14_Color.txt  
 extracting: data/obj/g15_Color.jpg  
  inflating: data/obj/g15_Color.txt  
 extracting: data/obj/g16_Color.jpg  
 extracting: data/obj/g16_Color.txt  
 extracting: data/obj/g17_Color.jpg  
 extracting: data/obj/g17_Color.txt  
 extracting: data/obj/g18_Color.jpg  
 extracting: data/obj/g18_Color.txt  
 extracting: data/obj/g19_Color.j

In [12]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/PXL_20221008_210010798.jpg', 'data/obj/g17_Color.jpg', 'data/obj/PXL_20221008_205844511.jpg', 'data/obj/g24_Color.jpg', 'data/obj/g10_Color.jpg', 'data/obj/PXL_20221008_212550892.jpg', 'data/obj/g22_Color.jpg', 'data/obj/PXL_20221008_205819519.jpg', 'data/obj/g5_Color.jpg', 'data/obj/g32_Color.jpg', 'data/obj/PXL_20221008_212640950.jpg', 'data/obj/g18_Color.jpg', 'data/obj/PXL_20221008_212752215.jpg', 'data/obj/PXL_20221008_212729542.jpg', 'data/obj/g12_Color.jpg', 'data/obj/g20_Color.jpg', 'data/obj/PXL_20221008_212719775.jpg', 'data/obj/PXL_20221008_205808872.jpg', 'data/obj/g28_Color.jpg', 'data/obj/g7_Color.jpg', 'data/obj/Gumby-4.jpg', 'data/obj/PXL_20221008_212620277.jpg', 'data/obj/g29_Color.jpg', 'data/obj/g16_Color.jpg', 'data/obj/clay-gumby-twisted.jpg', 'data/obj/g15_Color.jpg', 'data/obj/PXL_20221008_212553399.jpg', 'data/obj/PXL_20221008_212546971.jpg', 'data/obj/g0_Color.jpg', 'data/obj/15641_1.jpg', 'data/obj/g4_Color_Color.jpg', 'data/obj/PXL_20221008_2057359

In [13]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [14]:
#   only if starting from scratch
!wget https://pjreddie.com/media/files/darknet53.conv.74


--2022-10-09 17:29:40--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  16.9MB/s    in 10s     

2022-10-09 17:29:51 (15.2 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [15]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 350229, rewritten_bbox = 0.012563 % 
]2;3926/4000: loss=0.0 hours left=0.3
 3926: 0.025612, 0.035402 avg loss, 0.001000 rate, 9.940681 seconds, 251264 images, 0.290798 hours left
Loaded: 0.000054 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.896740), count: 5, class_loss = 0.011455, iou_loss = 0.038796, total_loss = 0.050251 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 350234, rewritten_bbox = 0.012563 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.896183), count: 6, class_loss = 0.005797, iou_loss = 0.042675, total_loss = 0.048472 
v3 (m

In [ ]:
%cp -v cfg/yolov3_training.cfg   /mydrive/yolov3

'cfg/yolov3_training.cfg' -> '/mydrive/yolov3/yolov3_training.cfg'


In [ ]:
!ls -lh  /mydrive/yolov3/yolov3_training_last_0.weights

-rw------- 1 root root 235M Oct  9 13:39 /mydrive/yolov3/yolov3_training_last_0.weights
